In [1]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from utils.extract import db_connection, download_data
import json
import numpy as np

%load_ext autoreload
%autoreload 2
# Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.

lista_tickers = pd.read_csv('tickers.csv')[['Ticker']]

now = datetime.datetime.now()

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2000-01-01'
end_date = '{}-{}-{}'.format(now.year, now.month, now.day)

with open('creds.txt', encoding='utf-8') as data_file:
    creds = json.loads(data_file.read())

In [2]:
end_date

'2018-9-17'

In [3]:
def create_new_stock_table(table_name, creds):
    """
    Hace una nueva tabla de stocks según el ticker que le pongas
    
    Args:
        table_name(str): nombre del ticker
        conn(connection object): objeto de connección a la base de datos
    """
    conn = db_connection(creds)
    query = """CREATE TABLE {} (id SERIAL,
    date date NOT NULL UNIQUE,
    high float,
    low float,
    open float,
    close float,
    volume float, 
    adj_close float, PRIMARY KEY (id));""".format(table_name)
    download_data(conn, query)

def update_in_db(df, table_name, creds):
    
    conn = db_connection(creds)
    matrix = np.array(df.to_records().view(type=np.matrix))[0]
    data = []
    
    for i in range(len(matrix)):
        date = "('" + str(matrix[i][0].year) + "-" + str(matrix[i][0].month) + "-" + str(matrix[i][0].day) + "')::date"
        High = str(matrix[i][1])
        Low = str(matrix[i][2])
        Open = str(matrix[i][3])
        Close = str(matrix[i][4])
        Volume = str(matrix[i][5])
        Adj_Clos = str(matrix[i][6])
        prices = "(" + date + ", " + High + ", " + Low + ", " + Open + ", " + Close + ", " + Volume + "," + Adj_Clos +")"
        data.append(prices)
    
    data = str(data).replace("[", "(").replace("]", ")").replace('(', '', 1)[:-1].replace('"','')
    table_name = table_name.replace('-','_')
    query = """INSERT INTO {} (date, high, low, open, close, volume, adj_close) VALUES{} ON CONFLICT ON CONSTRAINT {}_date_key DO NOTHING;""".format(table_name.upper(), data, table_name.lower())
    
    download_data(conn, query)

In [4]:
# User pandas_reader.data.DataReader to load the desired data. As simple as that.

for ticker in lista_tickers['Ticker']:
    panel_data = data.DataReader(ticker, 'yahoo', start_date, end_date)
    table_name = ticker.replace('.','_')
    create_new_stock_table(table_name, creds)
    update_in_db(panel_data, table_name, creds)
    print("Se guardó: {}".format(ticker))

ERROR:root:relation "xlk_mx" already exists



Se guardó: XLK.MX
Se guardó: XLE.MX
Se guardó: X.MX
Se guardó: WFC.MX
Se guardó: VGK.MX
Se guardó: V.MX
Se guardó: UNH.MX
Se guardó: TWTR.MX
Se guardó: TSLA.MX
Se guardó: T.MX
Se guardó: SLV.MX
Se guardó: SHY.MX
Se guardó: SBUX.MX
Se guardó: QQQ.MX
Se guardó: PFE.MX
Se guardó: NVDA.MX
Se guardó: NKE.MX
Se guardó: NFLX.MX
Se guardó: MU.MX
Se guardó: MSFT.MX
Se guardó: MCD.MX
Se guardó: MA.MX
Se guardó: JPM.MX
Se guardó: IVV.MX
Se guardó: IAU.MX
Se guardó: HD.MX
Se guardó: GOOGL.MX
Se guardó: GILD.MX
Se guardó: FB.MX
Se guardó: EMB.MX
Se guardó: EEM.MX
Se guardó: DIS.MX
Se guardó: CSCO.MX
Se guardó: COST.MX
Se guardó: BAC.MX
Se guardó: BABAN.MX
Se guardó: AXP.MX
Se guardó: AMZN.MX
Se guardó: AGNN.MX
Se guardó: AAPL.MX
Se guardó: WALMEX.MX
Se guardó: VINTE.MX
Se guardó: TERRA13.MX
Se guardó: SPORTS.MX
Se guardó: SORIANAB.MX
Se guardó: SMARTRC14.MX
Se guardó: RA.MX
Se guardó: OMAB.MX
Se guardó: NAFTRACISHRS.MX
Se guardó: MEXCHEM.MX
Se guardó: MEGACPO.MX


ERROR:root:syntax error at or near "-"
LINE 1: CREATE TABLE LIVEPOLC-1_MX (id SERIAL,
                             ^

ERROR:root:relation "livepolc_1_mx" does not exist
LINE 1: INSERT INTO LIVEPOLC_1_MX (date, high, low, open, close, vol...
                    ^



Se guardó: LIVEPOLC-1.MX
Se guardó: LALAB.MX
Se guardó: LACOMERUBC.MX
Se guardó: LABB.MX
Se guardó: KIMBERA.MX
Se guardó: GRUMAB.MX
Se guardó: GMEXICOB.MX
Se guardó: GFNORTEO.MX
Se guardó: GENTERA.MX
Se guardó: GAPB.MX
Se guardó: FUNO11.MX
Se guardó: FIBRAMQ12.MX
Se guardó: FEMSAUBD.MX
Se guardó: ELEKTRA.MX
Se guardó: DIABLOI10.MX
Se guardó: CUERVO.MX
Se guardó: CHDRAUIB.MX
Se guardó: CEMEXCPO.MX
Se guardó: CADUA.MX
Se guardó: BOLSAA.MX
Se guardó: BIMBOA.MX
Se guardó: BBAJIOO.MX
Se guardó: BACHOCOB.MX
Se guardó: ASURB.MX
Se guardó: ANGELD10.MX
Se guardó: AMXL.MX
Se guardó: ALSEA.MX
Se guardó: ALFAA.MX
Se guardó: AEROMEX.MX
Se guardó: ACTINVRB.MX
Se guardó: AC.MX
